In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_val, y_val = X_train[-10000:], y_train[-10000:]
X_train, y_train = X_train[:-10000], y_train[:-10000]

To load the MNIST dataset, we used the TensorFlow library, which provides a range of tools and libraries for machine learning and data analysis. 
We used the load_data function to load the MNIST data and split it into training, validation, and test sets.


Next, we will preprocess the data by normalizing the pixel values to be between 0 and 1. This is important because the quantum model we will use expects input data to be normalized. We will use the astype function to convert the data to floating point values and the reshape function to reshape the data into a 2D array.


In [4]:
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X_train = X_train.reshape(-1, 28*28)
X_val = X_val.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

#We also need to encode the labels as categorical variables using the to_categorical function from the tf.keras.utils module

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

Part 2: Defining a quantum function

In [5]:
!pip install pennylane --upgrade
import pennylane as qml
from pennylane import numpy as np
# Choose a device (e.g., 'default.qubit')
wires = [0,1,2,3,4,5,6,7,8,9]
device = qml.device('default.qubit',wires=[0,1,2,3,4,5,6,7,8,9])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
@qml.qnode(device)
def quantum_neural_net(weights, data):
  wires = range(10)
  # Initialize the qubits
  qml.templates.AmplitudeEmbedding(features=data, wires = wires, normalize=True,pad_with=0)
  #qml.templates.AmplitudeEmbedding(features=data, wires=wires, normalize=True,pad_with=0)
  # Apply multiple layers of quantum neurons
  for W in weights:
    qml.templates.StronglyEntanglingLayers(W, wires=wires)
  # Measure the qubits
  return [qml.expval(qml.PauliZ(i)) for i in wires]

#@qml.qnode(dev)
#def qnode(weights, inputs=None):
#qml.templates.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True)
#qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
#return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]



Part 3: Optimizing the model

In [7]:
# Define the optimization steps
steps = 100
learning_rate = 0.01
batch_size = 32
n_layers=5
n_wires=10
params = np.array(np.random.rand(n_layers,n_wires,10,3))
# Define the loss function
def loss(labels, predictions):
  predictions = tf.transpose(predictions, [1, 0])
  labels = tf.transpose(labels, [0, 1])
  return tf.losses.mean_squared_error(labels, predictions)

In [33]:
# Define the training loop
def train(X_train, y_train, X_val, y_val, epochs, opt):
  params = np.array(np.random.rand(n_layers,n_wires,10,3), requires_grad=True)
  for epoch in range(epochs):
    # Shuffle the training data
    indices = np.arange(len(X_train))
    np.random.shuffle(indices)
    X_train = X_train[indices]
    y_train = y_train[indices]
    # Split the training data into batches
    for i in range(0, len(X_train), batch_size):
      X_batch = X_train[i:i+batch_size]
      y_batch = y_train[i:i+batch_size]
      # Compute the loss and gradients
      with tf.GradientTape() as tape:
        predictions = quantum_neural_net(params, X_batch)
        loss_val = loss(y_batch, predictions)
        params = tf.convert_to_tensor(params)
        gradients = tape.gradient(loss_val, params)
        # Update the parameters, currently not working
        # step the optimizer to get the new values of weights and x
        #params = opt.step()
          #params = opt.step(gradients, params, x)
        # opt.apply_gradients(zip(gradients, X_batch))
    # Compute the loss and accuracy on the validation set
    predictions = quantum_neural_net(params, X_val)
    val_loss = loss(y_val, predictions)
    val_acc = qml.metrics.accuracy(y_val, predictions)
    # Print the progress
    print(f'Epoch {epoch+1}: loss = {val_loss:.4f}, accuracy = {val_acc:.4f}')


In [34]:
from pennylane.transforms import op_transforms
# Compute the accuracy on the test set
#shape = qml.StronglyEntanglingLayers.shape(n_layers=5, n_wires=10)
#weights = np.random.random(size=shape)
#weights=_process(weights)
#weights = np.array(np.random.random(shape*2), requires_grad=True)
#params = np.array(np.random.rand(n_layers,n_wires,10,3))
#qml.StronglyEntanglingLayers(weights, range(10))
opt = NesterovMomentumOptimizer()
#optimizers.Adam(learning_rate=0.01)
train(X_train, y_train, X_val, y_val, 2, opt)
predictions = quantum_neural_net(weights, X_test)
test_acc = qml.metrics.accuracy(y_test, predictions)
print(f'Test accuracy: {test_acc:.4f}')
  
  # Plot the predictions
plt.scatter(y_test, predictions)
plt.xlabel('True labels')
plt.ylabel('Predictions')
plt.show()

KeyboardInterrupt: ignored